<a href="https://colab.research.google.com/github/Azlan-Qaisrani/Langgraph/blob/main/DrafterAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langgraph langchain-core langgraph-prebuilt sentence-transformers faiss-cpu python-dotenv

In [2]:
!pip install langchain-google-genai

In [34]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage

from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI


In [35]:

import os
os.environ["GRPC_VERBOSITY"] = "NONE"
os.environ["GRPC_TRACE"] = ""
os.environ["GOOGLE_CLOUD_PROJECT"] = ""
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""
from google.colab import userdata


In [36]:
document_content= ""


In [37]:
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]



In [38]:
@tool
def update(content:str)-> str:
  """Updates the document with provided content."""
  global document_content
  document_content=content
  return f"Document hase been updated successfully! the curret content is :\n{document_content}"
@tool
def save(filename:str)-> str:
  """Save the current document to a text file and finish the process

  Args:
    filename:Name for the text file
    """
  global document_content
  if not filename.endswith(".txt"):
    filename=f"{filename}.txt"
  try:
    with open(filename,'w') as file:
      file.write(document_content)
    print(f"\n Document hase been saved to {filename}")
    return f"Document has been saved successfully to '{filename}'"
  except Exception as e:
    print(f"Error saving document: {e}")
    return f"Error saving document: {e}"
tools=[update,save]



In [39]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=userdata.get("google_api_key"),   # ← forces API key, no GCP
    temperature=0.7
).bind_tools(tools)

In [40]:
def our_agent(state: AgentState) -> AgentState:
    system_prompt = SystemMessage(content=f"""
    You are Drafter, a helpful writing assistant. You are going to help the user update and modify documents.

    - If the user wants to update or modify content, use the 'update' tool with the complete updated content.
    - If the user wants to save and finish, you need to use the 'save' tool.
    - Make sure to always show the current document state after modifications.

    The current document content is:{document_content}
    """)

    if not state["messages"]:
        user_input = "I'm ready to help you update a document. What would you like to create?"
        user_message = HumanMessage(content=user_input)

    else:
        user_input = input("\nWhat would you like to do with the document? ")
        print(f"\n👤 USER: {user_input}")
        user_message = HumanMessage(content=user_input)

    all_messages = [system_prompt] + list(state["messages"]) + [user_message]

    response = llm.invoke(all_messages)

    print(f"\n🤖 AI: {response.content}")
    if hasattr(response, "tool_calls") and response.tool_calls:
        print(f"🔧 USING TOOLS: {[tc['name'] for tc in response.tool_calls]}")

    return {"messages": list(state["messages"]) + [user_message, response]}


In [41]:
def should_continue(state:AgentState)->str:

  """Determine if we should continue or end the conversation."""
  messages=state['messages']
  if not messages:
    return "continue"

  for message in reversed(messages):
    if (isinstance(message,ToolMessage) and
        "saved" in message.content.lower() and
        "document" in message.content.lower()):
        return "end"
  return "continue"

In [42]:
def print_messages(messages):
    """Function I made to print the messages in a more readable format"""
    if not messages:
        return

    for message in messages[-3:]:
        if isinstance(message, ToolMessage):
            print(f"\nTOOL RESULT: {message.content}")





In [43]:
graph=StateGraph(AgentState)
graph.add_node("agent",our_agent)
graph.add_node("tools",ToolNode(tools))
graph.set_entry_point('agent')
graph.add_edge('agent','tools')
graph.add_conditional_edges(
    'tools',
    should_continue,
    {
        'continue':'agent',
        'end':END,
    },
)
app=graph.compile()

In [45]:
def run_document_agent():
    print("\n ===== DRAFTER =====")

    state = {"messages": []}

    for step in app.stream(state, stream_mode="values"):
        if "messages" in step:
            print_messages(step["messages"])

    print("\n ===== DRAFTER FINISHED =====")

if __name__ == "__main__":
    run_document_agent()


 ===== DRAFTER =====

🤖 AI: Okay, I understand. The current document content is:

Subject: Unable to Attend Meeting Today

Dear Tom,

I am writing to inform you that I will be unable to attend today's meeting because I am sick and currently in the hospital.

I apologize for any inconvenience this may cause. I will reach out to a colleague to get notes from the meeting and will catch up on any important updates as soon as possible.

Thank you for your understanding.

Sincerely,
Azlan

How would you like to update it?

What would you like to do with the document? can you add it that i will attend meeting tommoroow at 10pm if possible to secdule

👤 USER: can you add it that i will attend meeting tommoroow at 10pm if possible to secdule

🤖 AI: 
🔧 USING TOOLS: ['update']

TOOL RESULT: Document hase been updated successfully! the curret content is :
Subject: Unable to Attend Meeting Today

Dear Tom,

I am writing to inform you that I will be unable to attend today's meeting because I am sic